# Restaurant Recommender (Simplified)

This version only prints **Restaurant Name** and **Address** for a given city, cuisines, and price range.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('Dataset .csv')
df.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270


In [4]:
df['Cuisines'] = df['Cuisines'].fillna('Unknown')

def norm_text(s): return str(s).strip().lower()

df['city_clean'] = df['City'].apply(norm_text)
df['cuisines_clean'] = df['Cuisines'].apply(lambda x: ','.join([c.strip().lower() for c in str(x).split(',')]))
df['price_token'] = 'price_' + df['Price range'].astype(str)
df['combo_feature'] = df['city_clean'] + ' ' + df['cuisines_clean'] + ' ' + df['price_token']

vectorizer = CountVectorizer(ngram_range=(1,2), min_df=2)
X = vectorizer.fit_transform(df['combo_feature'])


In [10]:
def simple_recommender(city, cuisines, price_range, top_n=10):
    city_clean = str(city).strip().lower()
    cuisines_clean = ','.join([c.strip().lower() for c in str(cuisines).split(',')])
    price_token = 'price_' + str(price_range)
    query = city_clean + ' ' + cuisines_clean + ' ' + price_token
    q_vec = vectorizer.transform([query])
    scores = cosine_similarity(q_vec, X).ravel()
    ranked = np.argsort(-scores)
    rows = []
    for idx in ranked[:top_n]:
        rows.append({
            'Restaurant Name': df.loc[idx, 'Restaurant Name'],
            'Address': df.loc[idx, 'Address']
        })
    return pd.DataFrame(rows)



In [12]:
simple_recommender(city='New Delhi', cuisines='North Indian, Chinese', price_range=2, top_n=10)


,Restaurant Name,Address
0,Meenu Caterers,"C4F, DDA Market, Janakpuri, New Delhi"
1,The Viraj Food Zone,"61-B, Ber Sarai, JNU, New Delhi"
2,Brunchilli,"C-4B/295 A, Janakpuri, New Delhi"
3,Chic Fish,"32, Corner Market, Maharishi Dayanand Marg, Ma..."
4,Celebration Family Restaurant,"E-63, Near Sheetla Mata Mandir, Opposite Punja..."
5,Farratta,"17-18, 1st Floor, Corner Market, Malviya Nagar..."
6,Punjabi Handi,"G-2, Balaji Plaza, Bhera Enclave, LSC, Paschim..."
7,Gulnar Bar Be Que,"7-A/45, WEA, Karol Bagh, New Delhi"
8,Dial n Dine,"Shop 16, Ground Floor, DDA Market, Savita Viha..."
9,Handa's,"292, Near Radhu Palace Cinema, West Guru Angad..."
